### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [3]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(42)

model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.96155502 0.9694816  0.96897895 0.96642062 0.9667541  0.95513959
 0.9735076  0.97234978 0.97042616 0.97116628 0.960401   0.97068361
 0.97185653 0.97070134 0.97242774 0.95943363 0.97088414 0.97159111
 0.97017049 0.97210553 0.9569575  0.9697343  0.96686629 0.96785789
 0.96914825        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 15], Predictions: [0 1], [66  3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 15], Predictions: [0 1], [57 12]
Training RF ...
Model RF, Actual: [0 1], [54 15], Predictions: [0 1], [57 12]
Training XGB ...
Model XGB, Actual: [0 1], [54 15], Predictions: [0 1], [53 16]

None


[('ecg_mean', 243), ('bpm', 233), ('rmssd', 143), ('SCR_rate', 135), ('eda_mean', 84), ('sdnn', 72), ('ecg_median', 57), ('hf_rr', 48), ('lf_hf_ratio', 39), ('ecg_var', 36), ('ecg_

One or more of the test scores are non-finite: [0.85101766 0.85036833 0.85032045 0.85297659 0.85327455 0.85291554
 0.8661412  0.86790954 0.8636498  0.86318756 0.85518219 0.86062544
 0.85553434 0.85610516 0.85831145 0.86251267 0.86399241 0.86008531
 0.85492182 0.85949504 0.86222636 0.86304011 0.86100524 0.85582358
 0.86029734        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [66  4]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [61  9]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [62  8]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [61  9]

None


[('bpm', 98), ('ecg_mean', 79), ('ecg_std', 55), ('SCR_rate', 49), ('eda_mean', 48), ('ecg_median', 32), ('lf_hf_ratio', 26), ('sdnn', 19), ('mean_SCL', 18), ('ecg_var', 16), 

...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  8], Predictions: [0 1], [60 10]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  8], Predictions: [0 1], [62  8]
Training RF ...
Model RF, Actual: [0 1], [62  8], Predictions: [0 1], [62  8]
Training XGB ...
Model XGB, Actual: [0 1], [62  8], Predictions: [0 1], [62  8]

None


[('ecg_mean', 98), ('bpm', 94), ('SCR_rate', 52), ('hf_rr', 40), ('sdnn', 37), ('ecg_std', 34), ('ecg_median', 24), ('eda_mean', 23), ('ecg_var', 21), ('rmssd', 20), ('lf_hf_rat

One or more of the test scores are non-finite: [0.90350752 0.91271052 0.92533155 0.92288398 0.93165047 0.89910174
 0.91544354 0.92117027 0.92318872 0.93356048 0.90217561 0.92077133
 0.92732164 0.92639754 0.93538111 0.90966304 0.93374693 0.93519122
 0.93619488 0.94171973 0.91059734 0.929598   0.93880537 0.94131894
 0.94373578        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [65  5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [51 19]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [64  6]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [51 19]

None


[('ecg_mean', 226), ('bpm', 225), ('rmssd', 183), ('SCR_rate', 183), ('ecg_std', 110), ('sdnn', 93), ('eda_mean', 90), ('hf_rr', 83), ('lf_hf_ratio', 72), ('ecg_median', 57), ('mean

One or more of the test scores are non-finite: [0.91287871 0.92299622 0.91222575 0.92528525 0.92415517 0.90379914
 0.91742637 0.91029408 0.91863155 0.91993874 0.91660427 0.92102887
 0.91192304 0.91799313 0.91853857 0.91760848 0.90885709 0.90495565
 0.91015376 0.91084488 0.92913298 0.9163909  0.90717209 0.90590322
 0.90712574        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [56 14]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [51 19]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [50 20]

None


[('ecg_mean', 177), ('bpm', 154), ('SCR_rate', 93), ('ecg_std', 82), ('eda_mean', 72), ('lf_hf_ratio', 67), ('ecg_var', 52), ('ecg_median', 44), ('rmssd', 34), ('sdnn', 25), ('h

In [4]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(42)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=15)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.9903846153846154
	Precision: 1.0
	Recall: 0.9375
	F1-score: 0.967741935483871
	AUC score: 0.96875
----------------------------------------
